# Homework

### Dataset

In this homework, we'll build a model for predicting if we have an image of a dino or a dragon. For this, we will use the "Dino or Dragon?" dataset that can be downloaded from [Kaggle](https://www.kaggle.com/datasets/agrigorev/dino-or-dragon).

In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch.

Note: You don't need a computer with a GPU for this homework. A laptop or any personal computer should be sufficient.

### Data Preparation

The dataset contains around 1900 images of dinos and around 1900 images of dragons.

The dataset contains separate folders for training and validation.

### Model

For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

- The shape for input should be (150, 150, 3)

- Next, create a convolutional layer ([Conv2D](https://keras.io/api/layers/convolution_layers/convolution2d/)):

    - Use 32 filters

    - Kernel size should be (3, 3) (that's the size of the filter)

    - Use 'relu' as activation

- Reduce the size of the feature map with max pooling ([MaxPooling2D](https://keras.io/api/layers/pooling_layers/max_pooling2d/))

    - Set the pooling size to (2, 2)

- Turn the multi-dimensional result into vectors using a [Flatten](https://keras.io/api/layers/reshaping_layers/flatten/) layer

- Next, add a Dense layer with 64 neurons and 'relu' activation

- Finally, create the Dense layer with 1 neuron - this will be the output

    - The output layer should have an activation - use the appropriate activation for the binary classification case

- As optimizer use [SGD](https://keras.io/api/optimizers/sgd/) with the following parameters:

    - SGD(lr=0.002, momentum=0.8)
    
For clarification about kernel size and max pooling, check [Office Hours](https://www.youtube.com/watch?v=1WRgdBTUaAc).

[Homework](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/cohorts/2022/08-deep-learning/homework.md)

### Submit the results

Submit your results [here](https://forms.gle/XdH5ztBddvTvxzpT6): 

You can submit your solution multiple times. In this case, only the last submission will be used
If your answer doesn't match options exactly, select the closest one
Deadline

- The deadline for submitting is 21 November 2022, 23:00 CEST.

After that, the form will be closed.